## 高通滤波器（HPF）

- 检测图像的某个区域，然后根据像素与周围像素的亮度差值来提升该像素的亮度的滤波器
- kernel
[[0, -0.25, 0],
 [-0.25, 1, -0.25],
 [0, -0.25, 0]
]
- 核是指一组权重的集合，它会应用在原图像的一个区域，并由此生成目标图像的一个像素

In [1]:
import cv2
import numpy as np
from scipy import ndimage

- numpy只接受一位数组的卷积，素有选择用ndimage（Scipy模块中）的convolve()函数可以解决这个问题

In [2]:
# 生成两个滤波器，滤波器中的所有制加起来为0
kernel_3x3 = np.array([[-1, -1, -1],
                      [-1, 8, -1],
                      [-1, -1, -1]])
kernel_5x5 = np.array([[-1, -1, -1, -1, -1],
                       [-1, 1, 2, 1, -1],
                       [-1, 2, 4, 2, -1],
                       [-1, 1, 2, 1, -1],
                       [-1, -1, -1, -1,-1]])

In [3]:
img = cv2.imread('myPic.png', 0) # 以灰度图的形式读入
print(img.shape)

(1200, 1600)


In [4]:
k3 = ndimage.convolve(img, kernel_3x3) # 通过两个自定义的卷积核来实现两个高通滤波器
k5 = ndimage.convolve(img, kernel_5x5)

In [7]:
cv2.imshow("3x3", k3)
cv2.imshow("5x5", k5)

cv2.waitKey(10000)
cv2.destroyAllWindows()

In [8]:
cv2.imwrite('k3.png', k3)
cv2.imwrite('k5.png', k5)

True

## 低通滤波器（LPF）

- 在像素与周围像素的亮度差值小于一个特定值时，平滑该像素的亮度，主要用于去噪和模糊化，高斯模糊是最常用的模糊滤波器（平滑滤波器）之一，它是一个削弱高频信号强度的低通滤波器

In [ ]:
blured = cv2.GaussianBlur(img, (11, 11), 0) # 通过对图像应用低通滤波器之后，与原始图像计算差值，这种效果最好
g_hpf = img - blured

In [ ]:
cv2.imshow("g_hpf", g_hpf)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite('g_hpf.png',g_hpf)

## 边缘检测

- 1.OpenCV提供了许多边缘检测滤波函数，包括Laplacian()、Sobel()以及Scharr().
- 2.这些滤波函数都会将非边缘区域转化位黑色，将边缘区域转化为白色或其他饱和颜色.
- 3.但这些函数很容易将噪声错误的识别为边缘.
- 4.缓解这个问题的方法是在找到边缘之前对图像进行模糊处理.
- 5.opencv提供了许多模糊滤波函数，包括blur(简单算数平均),medianBlur()以及GaussianBlur（）
- 6.边缘检测滤波函数和模糊滤波函数的参数有很多，但总是会有一个ksize函数，是一个奇数，表示滤波核的宽度和高度（以像素为单位）
- 7.详细是实现见filter.py

- 1.使用medianBlur（）作为模糊函数，它对于去除数字化的视频噪声非常有效，特别是去除彩色图像的噪声
- 2.使用Laplacian（）函数作为边缘检测函数，会产生明显的边缘线条，灰度图像更是如此
- 3.在medianBlur（）之后，Laplacian（）之前，需将图像从BRG色彩空间转化为灰度色彩空间
- 4.得到Laplacian（)函数的结果后，需要将其转化为黑色边缘和白色背景的图像，然后将其归一化（使它的像素值在0-1之间），并乘以原图像以便能- 将边缘变黑

* 用定制内核做卷积
    - OpenCV提供了一个非常通用的filter2D()函数，它运用由用户指定的任意核或卷积矩阵

- cv2.filter2D(src, -1, kernel, dst)
- 第二个参数指定了目标图像的每个通道的深度（比如，位深度cv2.CV_8U表示每个通道为8位），如果是负值，则表示目标图像和原图像有相同的深度
- 对于彩色图像来说，filter2D()会对每个通道都用同样的核，如果要对每个通道使用不同的核，就必须用split（）函数和merge（）函数

## Canny边缘检测

- opencv还提供了一个非常简单的Canny函数，效果好，同时实现简单
- 1.高斯滤波器对图像进行去噪
- 2.计算梯度
- 3.在边缘上使用非最大值抑制
- 4.在检测到的边缘上使用双阈值去除假阳性
- 5.分析所用的边缘以及其之间的连接

In [5]:
img2 = cv2.imread('myPic.png', 0) # 以灰度图形式读入
data = (100, 300) # 需要自己设置传进去的参数
cv2.imwrite('canny.png', cv2.Canny(img2, *data))
# cv2.imwrite('canny.png', cv2.Canny(img2, 100, 300))

True

## 轮廓检测

- 物体检测（包括人脸）和物体目标跟踪中会大量使用

In [6]:
img3 = np.zeros((200, 200), dtype=np.uint8) # 创建一个200*200大小的黑色空白图像
img3[50:150, 50:150] = 255 # 在图像中央放置一个白色块

In [10]:
ret, thresh = cv2.threshold(img3, 127, 255, 0) # 二值化处理
# findContours()函数，有三个参数：输入图像、层次类型和轮廓逼近方法
print(ret)
print(thresh.shape)
image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# 色彩空间函数转换
color = cv2.cvtColor(img3, cv2.COLOR_GRAY2BGR)
# 画出轮廓，-1表示所有轮廓，画笔颜色为（（0， 255， 0）），即Green，粗细为2
img3 = cv2.drawContours(color, contours, -1, (0, 255, 0), 2)
cv2.imshow('contours', color)
cv2.waitKey(1000)
cv2.destroyAllWindows()

127.0
(200, 200, 3)


error: OpenCV(3.4.3) /io/opencv/modules/imgproc/src/contours.cpp:199: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'
